In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',max_iter=10)
df = pd.read_excel("transformeddata.xlsx")

In [2]:
df = df.fillna(df.mean())
df1 = df.fillna(method='ffill')
train, test = train_test_split(df1, test_size=0.2)

ysplit = df1['Vehicle Type']
xsplit = df1.drop(['Vehicle Type'],axis=1)
dt = xsplit.select_dtypes(include=['datetime64'])
cat2 = list(dt.columns)
categorical = xsplit.select_dtypes(exclude=['float64'])
cat3 = list(categorical.columns)
cols = cat2 + cat3
le = preprocessing.LabelEncoder()

In [3]:
xsplit = pd.get_dummies(xsplit, columns=cols)

In [4]:
bestfeatures = SelectKBest(score_func=chi2, k=110)
fit = bestfeatures.fit(xsplit,ysplit)

In [5]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(xsplit.columns)
fes = pd.concat([dfcolumns,dfscores],axis=1)
fes.columns = ['Specs','Score']
fes.nlargest(15,'Score')
fes = fes.sort_values(by =['Score'],ascending=False)
large = fes.head(10)

corr = large['Specs'].values
xsplit1 = xsplit[corr] 

In [6]:
xtrain, xtest = train_test_split(xsplit1, test_size=0.2)
ytrain, ytest = train_test_split(ysplit, test_size=0.2)
clf = clf.fit(xtrain,ytrain)
ypred = clf.predict(xtest)

In [7]:
result = accuracy_score(ytest,ypred)
print("The accuracy using logistic regression is {}".format(result))


The accuracy using logistic regression is 0.5535
